# R Serving with Plumber

## Dockerfile

* The Dockerfile defines the environment in which our server will be executed.
* Below, you can see that the entrypoint for our container will be [deploy.R](deploy.R)

In [ ]:
%pycat Dockerfile

## Code: deploy.R

The **deploy.R** script handles the following steps:
* Loads the R libraries used by the server.
* Loads a pretrained `xgboost` model that has been trained on the classical [Iris](https://archive.ics.uci.edu/ml/datasets/iris) dataset.
  * Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.
* Defines an inference function that takes a matrix of iris features and returns predictions for those iris examples.
* Finally, it imports the [endpoints.R](endpoints.R) script and launches the Plumber server app using those endpoint definitions.


In [ ]:
%pycat deploy.R

## Code: endpoints.R

**endpoints.R** defines two routes:
* `/ping` returns a string 'Alive' to indicate that the application is healthy
* `/invocations` applies the previously defined inference function to the input features from the request body

For more information about the requirements for building your own inference container, see:
[Use Your Own Inference Code with Hosting Services](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)

In [ ]:
%pycat endpoints.R

## Build the Serving Image

In [1]:
!docker build -t r-plumber .

Sending build context to Docker daemon  37.89kB
Step 1/8 : FROM r-base:4.1.1
 ---> 176db8b917ff
Step 2/8 : MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>
 ---> Using cache
 ---> 27f5aaa6d37a
Step 3/8 : RUN apt-get -y update && apt-get install -y --no-install-recommends     wget     apt-transport-https     ca-certificates     libcurl4-openssl-dev     libsodium-dev
 ---> Using cache
 ---> 646e8a8eb34f
Step 4/8 : RUN R -e "install.packages(c('plumber'), repos='https://cloud.r-project.org')"
 ---> Using cache
 ---> ad59e5a90079
Step 5/8 : COPY endpoints.R /opt/ml/endpoints.R
 ---> 2fc412bea09f
Step 6/8 : COPY deploy.R /opt/ml/deploy.R
 ---> edc847611640
Step 7/8 : WORKDIR /opt/ml
 ---> Running in 43c47f7c90c2
Removing intermediate container 43c47f7c90c2
 ---> e458fd783bb3
Step 8/8 : ENTRYPOINT ["/usr/bin/Rscript", "/opt/ml/deploy.R", "--no-save"]
 ---> Running in 3e6ca1515aa5
Removing intermediate container 3e6ca1515aa5
 ---> 06ba09c8dad6
Successfully built 06b

## Launch the Serving Container

In [2]:
!echo "Launching Plumber"
!docker run -d --rm -p 5000:8080 r-plumber
!echo "Waiting for the server to start.." && sleep 10

Launching Plumber
2c16319ca2c2f7fbd833038600ee3f8a7f1630b09db8a0fd6be8515476a5d709
docker: Error response from daemon: driver failed programming external connectivity on endpoint trusting_kilby (0aab9be97933e4767a5c65ce86ad93faf36ff070e22693348a22a4bdb979b28b): Bind for 0.0.0.0:5000 failed: port is already allocated.
Waiting for the server to start..


In [3]:
!docker container list

CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                    NAMES
3cbe00ff2de3        0d4d7e51047e        "/usr/bin/Rscript /o…"   3 hours ago         Up 3 hours          0.0.0.0:5000->8080/tcp   zen_knuth


## Define Simple Python Client

In [4]:
import requests
from tqdm import tqdm
import pandas as pd

pd.set_option("display.max_rows", 500)

In [5]:
def get_predictions(examples, instance=requests, port=5000):
    payload = {"features": examples}
    return instance.post(f"http://127.0.0.1:{port}/invocations", json=payload)

In [6]:
def get_health(instance=requests, port=5000):
    instance.get(f"http://127.0.0.1:{port}/ping")

## Define Example Inputs

Let's define example inputs from the Iris dataset.

In [7]:
x = [0]

### Plumber

In [10]:
predicted = get_predictions(x)

In [11]:
predicted.text

'[5.6549]'

### Push Image to ECR

In [ ]:
!./build_and_push.sh r-plumber

In [ ]:
# please copy the uri from ECR console.
image_name = "r-plumber"
# provide proviate the account id
account_id = ''
r_plumber_ecr_repo_uri = "{account_id}.dkr.ecr.ap-southeast-2.amazonaws.com/r-plumber"

### Create model and deploy on Endpoint

In [ ]:
import boto3
import sagemaker
from sagemaker.model import Model
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
model_name = 'my-r-model-sample-04' # must be unique
r_model = Model(image_uri = r_plumber_ecr_repo_uri, role = role, name = model_name)

In [ ]:
r_model.deploy(initial_instance_count = 1,
            instance_type = 'ml.m5.large'
              )

### Invoke endpoint

In [ ]:
client = boto3.client('sagemaker-runtime')

In [ ]:
x = [3, 4]

payload = str(x)

response = client.invoke_endpoint(
    EndpointName = 'my-r-model-sample-04-2021-09-08-02-01-06-589', # must be matched with the endpoint name
    Body = payload,
    ContentType='text/csv'
)

In [ ]:
response

In [ ]:
# response['Body'] is stream and can only be read once
result = response['Body'].read().decode()

In [ ]:
result

In [ ]:
print('output: ', result['output'])

In [ ]:
print('input:', x)

### Stop All Serving Containers

Finally, we will shut down the serving container we launched for the test.

In [ ]:
!docker kill $(docker ps -q)